# Widgets

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.vision import *
from fastai.widgets import DatasetFormatter, ImageCleaner, ImageDownloader

fastai offers several widgets to support the workflow of a deep learning practitioner. The purpose of the widgets are to help you organize, clean, and prepare your data for your model. Widgets are separated by data type.

## Images

### DatasetFormatter
The [`DatasetFormatter`](/widgets.image_cleaner.html#DatasetFormatter) class prepares your image dataset for widgets by returning a formatted [`DatasetTfm`](/vision.data.html#DatasetTfm) based on the [`DatasetType`](/basic_data.html#DatasetType) specified. Use `from_toplosses` to grab the most problematic images directly from your learner. Optionally, you can restrict the formatted dataset returned to `n_imgs`.

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
data.show_batch()

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.save('stage-1')

We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)

In [ ]:
db = (ImageItemList.from_folder(path)
                   .no_split()
                   .label_from_folder()
                   .databunch())

In [ ]:
learn = create_cnn(db, models.resnet18, metrics=[accuracy])
learn.load('stage-1');

### ImageCleaner

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) is for cleaning up images that don't belong in your dataset. It renders images in a row and gives you the opportunity to delete the file from your file system. To use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) we must first use `DatasetFormatter().from_toplosses` to get the suggested indices for misclassified images.

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn)

In [ ]:
ImageCleaner(ds, idxs, path)

### ImageDownloader

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) widget gives you a way to quickly bootstrap your image dataset without leaving the notebook. It searches and downloads images that match the search criteria and resolution / quality requirements and stores them on your filesystem within the provided `path`.

Images for each search query (or label) are stored in a separate folder within `path`. For example, if you pupulate `tiger` with a `path` setup to `./data`, you'll get a folder `./data/tiger/` with the tiger images in it.

In [ ]:
path = Path('./image_downloader_data')
ImageDownloader(path)

After populating images with [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader), you can get a an [`ImageDataBunch`](/vision.data.html#ImageDataBunch) by calling `ImageDataBunch.from_folder(path, size=size)`, or using the data block API.

In [ ]:
path.ls()

In [ ]:
src = (ImageItemList.from_folder(path)
       .random_split_by_pct()
       .label_from_folder()
       .transform(get_transforms(), size=224))
db  = src.databunch(bs=4)

In [ ]:
learn = create_cnn(db, models.resnet34, metrics=[accuracy])

In [ ]:
learn.fit(3)